# Mekanisme Perhatian dan Transformer

Satu kelemahan utama rangkaian berulang (recurrent networks) ialah semua perkataan dalam satu urutan mempunyai kesan yang sama terhadap hasilnya. Ini menyebabkan prestasi yang kurang optimum dengan model LSTM encoder-decoder standard untuk tugas urutan ke urutan, seperti Pengecaman Entiti Bernama dan Terjemahan Mesin. Dalam realiti, perkataan tertentu dalam urutan input sering kali mempunyai lebih banyak pengaruh terhadap output berurutan berbanding yang lain.

Pertimbangkan model urutan ke urutan, seperti terjemahan mesin. Ia dilaksanakan oleh dua rangkaian berulang, di mana satu rangkaian (**encoder**) akan memampatkan urutan input ke dalam keadaan tersembunyi, dan satu lagi, **decoder**, akan mengembangkan keadaan tersembunyi ini menjadi hasil terjemahan. Masalah dengan pendekatan ini ialah keadaan akhir rangkaian akan menghadapi kesukaran untuk mengingati permulaan ayat, menyebabkan kualiti model yang rendah pada ayat yang panjang.

**Mekanisme Perhatian** menyediakan cara untuk memberi pemberat kepada kesan kontekstual setiap vektor input terhadap setiap ramalan output RNN. Cara ia dilaksanakan adalah dengan mencipta jalan pintas antara keadaan perantaraan RNN input dan RNN output. Dengan cara ini, apabila menghasilkan simbol output $y_t$, kita akan mengambil kira semua keadaan tersembunyi input $h_i$, dengan pekali pemberat yang berbeza $\alpha_{t,i}$. 

![Imej menunjukkan model encoder/decoder dengan lapisan perhatian tambahan](../../../../../translated_images/ms/encoder-decoder-attention.7a726296894fb567.webp)
*Model encoder-decoder dengan mekanisme perhatian tambahan dalam [Bahdanau et al., 2015](https://arxiv.org/pdf/1409.0473.pdf), dipetik daripada [catatan blog ini](https://lilianweng.github.io/lil-log/2018/06/24/attention-attention.html)*

Matriks perhatian $\{\alpha_{i,j}\}$ akan mewakili tahap di mana perkataan input tertentu memainkan peranan dalam penjanaan perkataan tertentu dalam urutan output. Di bawah adalah contoh matriks seperti itu:

![Imej menunjukkan penjajaran sampel yang ditemui oleh RNNsearch-50, diambil daripada Bahdanau - arviz.org](../../../../../translated_images/ms/bahdanau-fig3.09ba2d37f202a6af.webp)

*Rajah diambil daripada [Bahdanau et al., 2015](https://arxiv.org/pdf/1409.0473.pdf) (Rajah 3)*

Mekanisme perhatian bertanggungjawab untuk sebahagian besar kemajuan terkini atau hampir terkini dalam Pemprosesan Bahasa Semula Jadi. Walau bagaimanapun, penambahan perhatian secara signifikan meningkatkan bilangan parameter model, yang membawa kepada isu penskalaan dengan RNN. Satu kekangan utama dalam penskalaan RNN ialah sifat berulang model ini menjadikannya mencabar untuk melatih secara kelompok dan selari. Dalam RNN, setiap elemen dalam urutan perlu diproses secara berurutan, yang bermaksud ia tidak boleh diselarikan dengan mudah.

Penggunaan mekanisme perhatian bersama kekangan ini membawa kepada penciptaan Model Transformer yang kini menjadi standard terkini yang kita kenali dan gunakan hari ini, daripada BERT hingga OpenGPT3.

## Model Transformer

Daripada meneruskan konteks setiap ramalan sebelumnya ke langkah penilaian seterusnya, **model transformer** menggunakan **pengekodan kedudukan** dan **perhatian** untuk menangkap konteks input tertentu dalam tetingkap teks yang disediakan. Imej di bawah menunjukkan bagaimana pengekodan kedudukan dengan perhatian dapat menangkap konteks dalam tetingkap tertentu.

![GIF animasi menunjukkan bagaimana penilaian dilakukan dalam model transformer.](../../../../../lessons/5-NLP/18-Transformers/images/transformer-animated-explanation.gif) 

Oleh kerana setiap kedudukan input dipetakan secara bebas ke setiap kedudukan output, transformer dapat diselarikan dengan lebih baik berbanding RNN, yang membolehkan model bahasa yang lebih besar dan lebih ekspresif. Setiap kepala perhatian boleh digunakan untuk mempelajari hubungan yang berbeza antara perkataan, yang meningkatkan tugas Pemprosesan Bahasa Semula Jadi hiliran.

## Membina Model Transformer Ringkas

Keras tidak mengandungi lapisan Transformer terbina dalam, tetapi kita boleh membinanya sendiri. Seperti sebelumnya, kita akan memberi tumpuan kepada pengelasan teks dataset AG News, tetapi perlu disebutkan bahawa model Transformer menunjukkan hasil terbaik pada tugas NLP yang lebih sukar.


In [1]:
import tensorflow as tf
from tensorflow import keras
import tensorflow_datasets as tfds
import numpy as np

ds_train, ds_test = tfds.load('ag_news_subset').values()

def extract_text(x):
    return x['title']+' '+x['description']

def tupelize(x):
    return (extract_text(x),x['label'])

Lapisan baharu dalam Keras perlu mewarisi kelas `Layer`, dan melaksanakan kaedah `call`. Mari kita mulakan dengan lapisan **Positional Embedding**. Kita akan menggunakan [beberapa kod daripada dokumentasi rasmi Keras](https://keras.io/examples/nlp/text_classification_with_transformer/). Kita akan menganggap bahawa kita melapik semua jujukan input kepada panjang `maxlen`.


In [2]:
class TokenAndPositionEmbedding(keras.layers.Layer):
    def __init__(self, maxlen, vocab_size, embed_dim):
        super(TokenAndPositionEmbedding, self).__init__()
        self.token_emb = keras.layers.Embedding(input_dim=vocab_size, output_dim=embed_dim)
        self.pos_emb = keras.layers.Embedding(input_dim=maxlen, output_dim=embed_dim)
        self.maxlen = maxlen

    def call(self, x):
        maxlen = self.maxlen
        positions = tf.range(start=0, limit=maxlen, delta=1)
        positions = self.pos_emb(positions)
        x = self.token_emb(x)
        return x+positions

Lapisan ini terdiri daripada dua lapisan `Embedding`: untuk pengekodan token (dengan cara yang telah kita bincangkan sebelum ini) dan kedudukan token. Kedudukan token dicipta sebagai urutan nombor semula jadi dari 0 hingga `maxlen` menggunakan `tf.range`, dan kemudian dihantar melalui lapisan embedding. Dua vektor embedding yang dihasilkan kemudian ditambah, menghasilkan representasi input yang tertanam secara kedudukan dengan bentuk `maxlen`$\times$`embed_dim`.

Sekarang, mari kita laksanakan blok transformer. Ia akan mengambil output daripada lapisan embedding yang telah ditakrifkan sebelum ini:


In [3]:
class TransformerBlock(keras.layers.Layer):
    def __init__(self, embed_dim, num_heads, ff_dim, rate=0.1):
        super(TransformerBlock, self).__init__()
        self.att = keras.layers.MultiHeadAttention(num_heads=num_heads, key_dim=embed_dim, name='attn')
        self.ffn = keras.Sequential(
            [keras.layers.Dense(ff_dim, activation="relu"), keras.layers.Dense(embed_dim),]
        )
        self.layernorm1 = keras.layers.LayerNormalization(epsilon=1e-6)
        self.layernorm2 = keras.layers.LayerNormalization(epsilon=1e-6)
        self.dropout1 = keras.layers.Dropout(rate)
        self.dropout2 = keras.layers.Dropout(rate)

    def call(self, inputs, training):
        attn_output = self.att(inputs, inputs)
        attn_output = self.dropout1(attn_output, training=training)
        out1 = self.layernorm1(inputs + attn_output)
        ffn_output = self.ffn(out1)
        ffn_output = self.dropout2(ffn_output, training=training)
        return self.layernorm2(out1 + ffn_output)

Transformer menggunakan `MultiHeadAttention` pada input yang telah dikodkan dengan posisi untuk menghasilkan vektor perhatian dengan dimensi `maxlen`$\times$`embed_dim`, yang kemudian dicampur dengan input dan dinormalisasi menggunakan `LayerNormalization`.

> **Nota**: `LayerNormalization` adalah serupa dengan `BatchNormalization` yang dibincangkan dalam bahagian *Computer Vision* dalam laluan pembelajaran ini, tetapi ia menormalkan output lapisan sebelumnya untuk setiap sampel latihan secara bebas, untuk membawanya ke julat [-1..1].

Output daripada lapisan ini kemudian dihantar melalui rangkaian `Dense` (dalam kes kita - perceptron dua lapisan), dan hasilnya ditambah kepada output akhir (yang sekali lagi menjalani normalisasi). 

Sekarang, kita bersedia untuk mentakrifkan model transformer lengkap:


In [4]:
embed_dim = 32  # Embedding size for each token
num_heads = 2  # Number of attention heads
ff_dim = 32  # Hidden layer size in feed forward network inside transformer
maxlen = 256
vocab_size = 20000

model = keras.models.Sequential([
    keras.layers.experimental.preprocessing.TextVectorization(max_tokens=vocab_size,output_sequence_length=maxlen, input_shape=(1,)),
    TokenAndPositionEmbedding(maxlen, vocab_size, embed_dim),
    TransformerBlock(embed_dim, num_heads, ff_dim),
    keras.layers.GlobalAveragePooling1D(),
    keras.layers.Dropout(0.1),
    keras.layers.Dense(20, activation="relu"),
    keras.layers.Dropout(0.1),
    keras.layers.Dense(4, activation="softmax")
])

model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
text_vectorization (TextVect (None, 256)               0         
_________________________________________________________________
token_and_position_embedding (None, 256, 32)           648192    
_________________________________________________________________
transformer_block (Transform (None, 256, 32)           10656     
_________________________________________________________________
global_average_pooling1d (Gl (None, 32)                0         
_________________________________________________________________
dropout_2 (Dropout)          (None, 32)                0         
_________________________________________________________________
dense_2 (Dense)              (None, 20)                660       
_________________________________________________________________
dropout_3 (Dropout)          (None, 20)               

In [5]:
print('Training tokenizer')
model.layers[0].adapt(ds_train.map(extract_text))
model.compile(loss='sparse_categorical_crossentropy',metrics=['acc'], optimizer='adam')
model.fit(ds_train.map(tupelize).batch(128),validation_data=ds_test.map(tupelize).batch(128))

Training tokenizer
938/938 [==============================] - 45s 39ms/step - loss: 0.4978 - acc: 0.8068 - val_loss: 0.2808 - val_acc: 0.9124


## Model Transformer BERT

**BERT** (Bidirectional Encoder Representations from Transformers) ialah rangkaian transformer berbilang lapisan yang sangat besar dengan 12 lapisan untuk *BERT-base*, dan 24 lapisan untuk *BERT-large*. Model ini mula-mula dilatih awal menggunakan korpus teks yang besar (WikiPedia + buku) melalui latihan tanpa pengawasan (meramalkan perkataan yang disembunyikan dalam ayat). Semasa latihan awal, model ini menyerap tahap pemahaman bahasa yang signifikan yang kemudiannya boleh dimanfaatkan dengan dataset lain melalui penalaan halus. Proses ini dipanggil **pembelajaran pemindahan**.

![gambar dari http://jalammar.github.io/illustrated-bert/](../../../../../translated_images/ms/jalammarBERT-language-modeling-masked-lm.34f113ea5fec4362.webp)

Terdapat banyak variasi seni bina Transformer termasuk BERT, DistilBERT, BigBird, OpenGPT3 dan banyak lagi yang boleh ditala halus.

Mari kita lihat bagaimana kita boleh menggunakan model BERT yang telah dilatih awal untuk menyelesaikan masalah klasifikasi jujukan tradisional kita. Kita akan meminjam idea dan beberapa kod daripada [dokumentasi rasmi](https://www.tensorflow.org/text/tutorials/classify_text_with_bert).

Untuk memuatkan model yang telah dilatih awal, kita akan menggunakan **Tensorflow hub**. Pertama, mari kita muatkan penvektor khusus BERT:


In [1]:
import tensorflow_text 
import tensorflow_hub as hub
vectorizer = hub.KerasLayer('https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3')

ModuleNotFoundError: No module named 'tensorflow_text'

In [7]:
vectorizer(['I love transformers'])

{'input_type_ids': <tf.Tensor: shape=(1, 128), dtype=int32, numpy=
 array([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]],
       dtype=int32)>,
 'input_word_ids': <tf.Tensor: shape=(1, 128), dtype=int32, numpy=
 array([[  101,  1045,  2293, 19081,   102,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0, 

Adalah penting untuk menggunakan penvektor yang sama seperti yang digunakan oleh rangkaian asal semasa dilatih. Selain itu, penvektor BERT mengembalikan tiga komponen:
* `input_word_ids`, iaitu satu urutan nombor token untuk ayat input
* `input_mask`, menunjukkan bahagian mana dalam urutan yang mengandungi input sebenar, dan bahagian mana yang merupakan padding. Ia serupa dengan topeng yang dihasilkan oleh lapisan `Masking`
* `input_type_ids` digunakan untuk tugas pemodelan bahasa, dan membolehkan untuk menentukan dua ayat input dalam satu urutan.

Kemudian, kita boleh memulakan pengekstrak ciri BERT:


In [8]:
bert = hub.KerasLayer('https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-4_H-128_A-2/1')

In [9]:
z = bert(vectorizer(['I love transformers']))
for i,x in z.items():
    print(f"{i} -> { len(x) if isinstance(x, list) else x.shape }")

pooled_output -> (1, 128)
encoder_outputs -> 4
sequence_output -> (1, 128, 128)
default -> (1, 128)


Jadi, lapisan BERT mengembalikan beberapa hasil yang berguna:
* `pooled_output` adalah hasil daripada purata semua token dalam urutan. Anda boleh menganggapnya sebagai embedding semantik pintar untuk keseluruhan rangkaian. Ia setara dengan output lapisan `GlobalAveragePooling1D` dalam model kita sebelum ini.
* `sequence_output` adalah output daripada lapisan transformer terakhir (bersamaan dengan output `TransformerBlock` dalam model kita di atas).
* `encoder_outputs` adalah output daripada semua lapisan transformer. Oleh kerana kita telah memuatkan model BERT 4-lapisan (seperti yang mungkin anda teka daripada namanya, yang mengandungi `4_H`), ia mempunyai 4 tensor. Tensor terakhir adalah sama seperti `sequence_output`.

Sekarang kita akan mentakrifkan model klasifikasi hujung-ke-hujung. Kita akan menggunakan *definisi model fungsional*, di mana kita mentakrifkan input model, dan kemudian menyediakan beberapa ekspresi untuk mengira outputnya. Kita juga akan menjadikan berat model BERT tidak boleh dilatih, dan hanya melatih pengklasifikasi terakhir:


In [10]:
inp = keras.Input(shape=(),dtype=tf.string)
x = vectorizer(inp)
x = bert(x)
x = keras.layers.Dropout(0.1)(x['pooled_output'])
out = keras.layers.Dense(4,activation='softmax')(x)
model = keras.models.Model(inp,out)
bert.trainable = False
model.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None,)]            0                                            
__________________________________________________________________________________________________
keras_layer (KerasLayer)        {'input_type_ids': ( 0           input_1[0][0]                    
__________________________________________________________________________________________________
keras_layer_1 (KerasLayer)      {'pooled_output': (N 4782465     keras_layer[0][0]                
                                                                 keras_layer[0][1]                
                                                                 keras_layer[0][2]                
______________________________________________________________________________________________

In [11]:
model.compile(loss='sparse_categorical_crossentropy',metrics=['acc'], optimizer='adam')
model.fit(ds_train.map(tupelize).batch(128),validation_data=ds_test.map(tupelize).batch(128))

938/938 [==============================] - 528s 559ms/step - loss: 0.8056 - acc: 0.6983 - val_loss: 0.5953 - val_acc: 0.7888


Walaupun terdapat sedikit parameter yang boleh dilatih, prosesnya agak perlahan kerana pengekstrak ciri BERT memerlukan pengiraan yang berat. Nampaknya kita tidak dapat mencapai ketepatan yang munasabah, sama ada disebabkan kekurangan latihan atau kekurangan parameter model.

Mari kita cuba membuka pembekuan berat BERT dan melatihnya juga. Ini memerlukan kadar pembelajaran yang sangat kecil, serta strategi latihan yang lebih teliti dengan **warmup**, menggunakan pengoptimum **AdamW**. Kita akan menggunakan pakej `tf-models-official` untuk mencipta pengoptimum:


In [12]:
from official.nlp import optimization 
bert.trainable=True
model.summary()
epochs = 3
opt = optimization.create_optimizer(
    init_lr=3e-5,
    num_train_steps=epochs*len(ds_train),
    num_warmup_steps=0.1*epochs*len(ds_train),
    optimizer_type='adamw')

model.compile(loss='sparse_categorical_crossentropy',metrics=['acc'], optimizer=opt)
model.fit(ds_train.map(tupelize).batch(128),validation_data=ds_test.map(tupelize).batch(128))

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None,)]            0                                            
__________________________________________________________________________________________________
keras_layer (KerasLayer)        {'input_type_ids': ( 0           input_1[0][0]                    
__________________________________________________________________________________________________
keras_layer_1 (KerasLayer)      {'pooled_output': (N 4782465     keras_layer[0][0]                
                                                                 keras_layer[0][1]                
                                                                 keras_layer[0][2]                
______________________________________________________________________________________________

Seperti yang anda lihat, latihan berjalan agak perlahan - tetapi anda mungkin ingin mencuba dan melatih model selama beberapa epoch (5-10) dan lihat jika anda boleh mendapatkan hasil terbaik berbanding pendekatan yang telah kita gunakan sebelum ini.

## Perpustakaan Huggingface Transformers

Satu lagi cara yang sangat biasa (dan sedikit lebih mudah) untuk menggunakan model Transformer adalah [pakej HuggingFace](https://github.com/huggingface/), yang menyediakan blok binaan mudah untuk pelbagai tugas NLP. Ia tersedia untuk Tensorflow dan PyTorch, satu lagi rangka kerja rangkaian neural yang sangat popular.

> **Nota**: Jika anda tidak berminat untuk melihat bagaimana perpustakaan Transformers berfungsi - anda boleh melangkau ke penghujung notebook ini, kerana anda tidak akan melihat apa-apa yang sangat berbeza daripada apa yang telah kita lakukan di atas. Kita akan mengulangi langkah-langkah yang sama untuk melatih model BERT menggunakan perpustakaan yang berbeza dan model yang jauh lebih besar. Oleh itu, proses ini melibatkan latihan yang agak panjang, jadi anda mungkin hanya ingin melihat kodnya.

Mari kita lihat bagaimana masalah kita boleh diselesaikan menggunakan [Huggingface Transformers](http://huggingface.co).


Perkara pertama yang perlu kita lakukan ialah memilih model yang akan kita gunakan. Selain beberapa model terbina dalam, Huggingface mempunyai [repositori model dalam talian](https://huggingface.co/models), di mana anda boleh menemui banyak lagi model pra-latihan daripada komuniti. Semua model tersebut boleh dimuatkan dan digunakan hanya dengan menyediakan nama model. Semua fail binari yang diperlukan untuk model tersebut akan dimuat turun secara automatik.

Pada masa tertentu, anda mungkin perlu memuatkan model anda sendiri, dalam hal ini anda boleh menentukan direktori yang mengandungi semua fail berkaitan, termasuk parameter untuk tokenizer, fail `config.json` dengan parameter model, berat binari, dan sebagainya.

Daripada nama model, kita boleh mencipta kedua-dua model dan tokenizer. Mari kita mulakan dengan tokenizer:


In [2]:
import transformers

# To load the model from Internet repository using model name. 
# Use this if you are running from your own copy of the notebooks
bert_model = 'bert-base-uncased' 

# To load the model from the directory on disk. Use this for Microsoft Learn module, because we have
# prepared all required files for you.
#bert_model = './bert'

tokenizer = transformers.BertTokenizer.from_pretrained(bert_model)

MAX_SEQ_LEN = 128
PAD_INDEX = tokenizer.convert_tokens_to_ids(tokenizer.pad_token)
UNK_INDEX = tokenizer.convert_tokens_to_ids(tokenizer.unk_token)

Objek `tokenizer` mengandungi fungsi `encode` yang boleh digunakan secara langsung untuk mengekod teks:


In [3]:
tokenizer.encode('Tensorflow is a great framework for NLP')

[101, 23435, 12314, 2003, 1037, 2307, 7705, 2005, 17953, 2361, 102]

Kita juga boleh menggunakan pengekod token untuk mengekod satu jujukan dengan cara yang sesuai untuk dihantar kepada model, iaitu termasuk medan `token_ids`, `input_mask`, dan sebagainya. Kita juga boleh menentukan bahawa kita mahu tensor Tensorflow dengan menyediakan argumen `return_tensors='tf'`:


In [4]:
tokenizer(['Hello, there'],return_tensors='tf')

{'input_ids': <tf.Tensor: shape=(1, 5), dtype=int32, numpy=array([[ 101, 7592, 1010, 2045,  102]], dtype=int32)>, 'token_type_ids': <tf.Tensor: shape=(1, 5), dtype=int32, numpy=array([[0, 0, 0, 0, 0]], dtype=int32)>, 'attention_mask': <tf.Tensor: shape=(1, 5), dtype=int32, numpy=array([[1, 1, 1, 1, 1]], dtype=int32)>}

Dalam kes kita, kita akan menggunakan model BERT yang telah dilatih terlebih dahulu yang dipanggil `bert-base-uncased`. *Uncased* menunjukkan bahawa model ini tidak sensitif kepada huruf besar atau kecil.

Semasa melatih model, kita perlu menyediakan urutan yang telah ditokenkan sebagai input, dan oleh itu kita akan mereka bentuk saluran pemprosesan data. Oleh kerana `tokenizer.encode` adalah fungsi Python, kita akan menggunakan pendekatan yang sama seperti dalam unit sebelumnya dengan memanggilnya menggunakan `py_function`:


In [31]:
def process(x):
    return tokenizer.encode(x.numpy().decode('utf-8'),return_tensors='tf',padding='max_length',max_length=MAX_SEQ_LEN,truncation=True)[0]

def process_fn(x):
    s = x['title']+' '+x['description']
    e = tf.py_function(process,inp=[s],Tout=(tf.int32))
    e.set_shape(MAX_SEQ_LEN)
    return e,x['label']

Sekarang kita boleh memuatkan model sebenar menggunakan pakej `BertForSequenceClassfication`. Ini memastikan bahawa model kita sudah mempunyai seni bina yang diperlukan untuk klasifikasi, termasuk pengklasifikasi akhir. Anda akan melihat mesej amaran yang menyatakan bahawa berat pengklasifikasi akhir tidak diinisialisasi, dan model memerlukan pra-latihan - itu adalah perkara yang biasa, kerana itulah yang akan kita lakukan!


In [32]:
model = transformers.TFBertForSequenceClassification.from_pretrained(bert_model,num_labels=4,output_attentions=False)

In [33]:
model.summary()

Model: "tf_bert_for_sequence_classification_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
bert (TFBertMainLayer)       multiple                  109482240 
_________________________________________________________________
dropout_75 (Dropout)         multiple                  0         
_________________________________________________________________
classifier (Dense)           multiple                  3076      
Total params: 109,485,316
Trainable params: 109,485,316
Non-trainable params: 0
_________________________________________________________________


Seperti yang anda boleh lihat daripada `summary()`, model ini mengandungi hampir 110 juta parameter! Mungkin, jika kita mahu tugas klasifikasi mudah pada set data yang agak kecil, kita tidak mahu melatih lapisan asas BERT:


In [34]:
model.layers[0].trainable = False
model.summary()

Model: "tf_bert_for_sequence_classification_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
bert (TFBertMainLayer)       multiple                  109482240 
_________________________________________________________________
dropout_75 (Dropout)         multiple                  0         
_________________________________________________________________
classifier (Dense)           multiple                  3076      
Total params: 109,485,316
Trainable params: 3,076
Non-trainable params: 109,482,240
_________________________________________________________________


Sekarang kita bersedia untuk memulakan latihan!

> **Nota**: Melatih model BERT berskala penuh boleh mengambil masa yang sangat lama! Oleh itu, kita hanya akan melatihnya untuk 32 batch pertama. Ini hanya untuk menunjukkan bagaimana latihan model disediakan. Jika anda berminat untuk mencuba latihan berskala penuh - hanya keluarkan parameter `steps_per_epoch` dan `validation_steps`, dan bersedialah untuk menunggu!


In [30]:
model.compile('adam','sparse_categorical_crossentropy',['acc'])
tf.get_logger().setLevel('ERROR')
model.fit(ds_train.map(process_fn).batch(32),validation_data=ds_test.map(process_fn).batch(32),steps_per_epoch=32,validation_steps=2)

32/32 [==============================] - 142s 4s/step - loss: 1.3896 - acc: 0.2500 - val_loss: 1.3863 - val_acc: 0.2480


Jika anda meningkatkan bilangan iterasi dan menunggu cukup lama, serta melatih untuk beberapa epoch, anda boleh menjangkakan bahawa klasifikasi BERT memberikan ketepatan terbaik! Ini kerana BERT sudah memahami struktur bahasa dengan baik, dan kita hanya perlu menyesuaikan pengklasifikasi akhir. Walau bagaimanapun, kerana BERT adalah model yang besar, keseluruhan proses latihan mengambil masa yang lama dan memerlukan kuasa pengkomputeran yang tinggi! (GPU, dan sebaiknya lebih daripada satu).

> **Note:** Dalam contoh kami, kami telah menggunakan salah satu model BERT pra-latih yang paling kecil. Terdapat model yang lebih besar yang berkemungkinan memberikan hasil yang lebih baik.


## Perkara Penting

Dalam unit ini, kita telah melihat seni bina model terkini berdasarkan **transformer**. Kita telah menggunakannya untuk tugas klasifikasi teks, tetapi model BERT juga boleh digunakan untuk pengekstrakan entiti, menjawab soalan, dan tugas NLP lain.

Model transformer mewakili tahap terkini dalam NLP, dan dalam kebanyakan kes, ia seharusnya menjadi penyelesaian pertama yang anda cuba apabila melaksanakan penyelesaian NLP tersuai. Walau bagaimanapun, memahami prinsip asas rangkaian neural berulang yang dibincangkan dalam modul ini adalah sangat penting jika anda ingin membina model neural yang lebih maju.



---

**Penafian**:  
Dokumen ini telah diterjemahkan menggunakan perkhidmatan terjemahan AI [Co-op Translator](https://github.com/Azure/co-op-translator). Walaupun kami berusaha untuk memastikan ketepatan, sila ambil perhatian bahawa terjemahan automatik mungkin mengandungi kesilapan atau ketidaktepatan. Dokumen asal dalam bahasa asalnya harus dianggap sebagai sumber yang berwibawa. Untuk maklumat yang kritikal, terjemahan manusia profesional adalah disyorkan. Kami tidak bertanggungjawab atas sebarang salah faham atau salah tafsir yang timbul daripada penggunaan terjemahan ini.
